<font size="5">Create SNR data set with all files</font>

Create a data set with a certain SNR. But now include all files even files that are shorter than one second. 

In [1]:
# Import modules and dependencies

import os
from os import listdir
from os.path import isdir, join
import pathlib
import shutil

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
from tensorflow.python.ops import gen_audio_ops as audio_ops


import akida
import akida_models
from akida_models import kws
from akida_models.kws import preprocessing
import cnn2snn
from cnn2snn import check_model_compatibility
from cnn2snn import quantize
from cnn2snn import quantize_layer
from cnn2snn import convert

import librosa
import scipy
from scipy import io
from scipy.io import wavfile
from scipy.io.wavfile import read
import python_speech_features
import random

<font size="5">1. Load the data set</font>

The Google Speech Command data set is loaded in this section of the code as a randomly ordered list of directories.

In [2]:
# If needed, set seed for experiment reproducibility

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# Define directories of the data
# Folder Silence only includes the silence category

data_dir = pathlib.Path('data/Modded_Google')
silence_dir = pathlib.Path('data/Modded_Google/silence')

In [4]:
# Check if the keywords are the desired ones

targets = np.array(tf.io.gfile.listdir(str(data_dir)))

targets = targets[targets != 'README.md']

print('Known and unknown commands:', targets)

Known and unknown commands: ['off' 'up' 'down' 'on' 'stop' 'yes' 'right' 'unknown' 'left' 'go' 'no'
 'silence']


In [5]:
# Create list of file names, along with its target 

filenames = []
y = []
for i, target in enumerate(targets):
    print(join(data_dir, target))
    filenames.append(listdir(join(data_dir, target)))
    y.append(np.ones(len(filenames[i])) * i)

data/Modded_Google/off
data/Modded_Google/up
data/Modded_Google/down
data/Modded_Google/on
data/Modded_Google/stop
data/Modded_Google/yes
data/Modded_Google/right
data/Modded_Google/unknown
data/Modded_Google/left
data/Modded_Google/go
data/Modded_Google/no
data/Modded_Google/silence


In [6]:
# Flatten the array to make it a list instead of a list of arrays

silence_filenames = []
silence_filenames.append(listdir(silence_dir))
print(np.shape(silence_filenames))

(1, 402)


In [7]:
# Flatten the array to make it a list instead of a list of arrays

filenames = [item for sublist in filenames for item in sublist]
y = [item for sublist in y for item in sublist]
print('Filenames: ', len(filenames))
print('y: ', len(y))

Filenames:  106231
y:  106231


In [8]:
# Distribution of all labels

unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0.0: 3745,
 1.0: 3723,
 2.0: 3917,
 3.0: 3845,
 4.0: 3872,
 5.0: 4044,
 6.0: 3778,
 7.0: 67283,
 8.0: 3801,
 9.0: 3880,
 10.0: 3941,
 11.0: 402}

In [9]:
# There are 38546 files that are not unkown or silence 
# Create number that should be equal to 10 % of the final data set
# This number is later used, such that the category "silence" and "unkown" make up 10% of the data.

amount_sil_un = 38546 / 0.8
amount_sil_un = amount_sil_un * 0.1
print(int(amount_sil_un))


4818


In [10]:
# Make file names of silence to one long list 

silence_filenames = np.squeeze(silence_filenames)

# Print the length in order to check if something has changed

print(len(silence_filenames))

402


In [11]:
# Include only the silence files which have a total of one second in length

silence_temp = []


for index, filename in enumerate(silence_filenames):

    path = join(silence_dir, filename)
    wav_loader = tf.io.read_file(path)
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    
    if wav_decoder[0].shape == (16000,1):
        silence_temp.append(filename)

silence_filenames = silence_temp

print('Silence Filenames: ', len(silence_filenames))

2022-04-24 10:32:06.732692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:06.780389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:06.780566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:06.781580: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Silence Filenames:  398


2022-04-24 10:32:06.781980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:06.782118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:06.782241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:07.107716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 10:32:07.107888: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [12]:
# Shuffle file names of the silence category
# Print the first position of the list to check if it worked

print(silence_filenames[0])
random.shuffle(silence_filenames)
print(silence_filenames[0])

31dude_miaowing.wav
15running_tap.wav


In [13]:
# Randomly select files of the "unknown" category in order to match the 10% constraint

filenames_target = list(zip(filenames, y))
random.shuffle(filenames_target)
#print(filenames_target)
counter = 0
index = 0
unknown_id = 7
result = []
for i in filenames_target:
    if filenames_target[index][1] != unknown_id:
        result.append(i)
    if filenames_target[index][1] == unknown_id:
        counter += 1
        if counter <= amount_sil_un:
            result.append(i)
    index += 1
filenames_target = result
    
print(len(filenames_target))
#print(filenames_target)

43766


In [14]:
# Check if the distribution still matches the one from the beginning.

filenames, y = zip(*filenames_target)
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0.0: 3745,
 1.0: 3723,
 2.0: 3917,
 3.0: 3845,
 4.0: 3872,
 5.0: 4044,
 6.0: 3778,
 7.0: 4818,
 8.0: 3801,
 9.0: 3880,
 10.0: 3941,
 11.0: 402}

In [15]:
# Duplicate silence files in random order till it is 10% of the data
# The randomness was introduced by previously shuffling the set

counter = 402
silence_id = 11
result = filenames_target
index = 0
amount_sil_un = 4818

while counter < amount_sil_un:
    for i in filenames_target:
        if filenames_target[index][1] == silence_id:
            if counter < amount_sil_un:
                counter +=1
                result.append(i)
                
        index += 1
    
    

filenames_target = result
#print(counter)
#print(len(filenames_target))
#print(filenames_target)

In [16]:
# Shuffle filenames again and unzip

#filenames_target = list(zip(filenames, y))
random.shuffle(filenames_target)
filenames, y = zip(*filenames_target)

# Check if data has the right length

print(len(filenames))

48182


In [17]:
# Check if nothing changed besideds "unknown" and "silence"

unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0.0: 3745,
 1.0: 3723,
 2.0: 3917,
 3.0: 3845,
 4.0: 3872,
 5.0: 4044,
 6.0: 3778,
 7.0: 4818,
 8.0: 3801,
 9.0: 3880,
 10.0: 3941,
 11.0: 4818}

In [18]:
# Indexing such that no file gets left out due to the integer precision

num_samples = (len(filenames_target))

a = num_samples * 0.8
b = num_samples * 0.1
c = num_samples - int(a) - int(b) - int(b)
int(a)
int(b)
print('80% : ', int(a))
print('10% : ', int(b))
print('Number of total samples: ', num_samples)
print('80% + 10% + 10% : ', int(a) + int(b) + int(b))
print('Number of samples that would be left out : ', int(c))

80% :  38545
10% :  4818
Number of total samples:  48182
80% + 10% + 10% :  48181
Number of samples that would be left out :  1


In [19]:
# Split and print all sets

train_files = filenames[:int(a)+int(c)]
val_files = filenames[int(a)+int(c): int(a)+int(c) + int(b)]
test_files = filenames[-int(b):]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

Training set size 38546
Validation set size 4818
Test set size 4818


In [20]:
# Same splitting procedure for the labels

y_train = y[:int(a)+int(c)]
y_val = y[int(a)+int(c): int(a)+int(c) + int(b)]
y_test = y[-int(b):]

print('Training set size', len(y_train))
print('Validation set size', len(y_val))
print('Test set size', len(y_test))

Training set size 38546
Validation set size 4818
Test set size 4818


<font size="5">2. Define MFCC functions </font>

In this chapter the functions for calculating the MFCCs are defined.

In [21]:
 model_settings = akida_models.kws.preprocessing.prepare_model_settings(sample_rate = 16000, clip_duration_ms = 1000, window_size_ms = 30, window_stride_ms = 10, feature_bin_count = 40)
print(model_settings)

{'desired_samples': 16000, 'window_size_samples': 480, 'window_stride_samples': 160, 'spectrogram_length': 98, 'fingerprint_width': 40, 'fingerprint_size': 3920}


In [26]:
# Function: Create MFCC from given path
# decoder_wave now specifies desired length instead of using the actul length "-1"
# decode_wav simply pads the audio files with 0 to the desired length

def calc_mfcc(path,num_ceps):
    
    # Load wavefile
    
    wav_loader = tf.io.read_file(path)
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    #wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    if wav_decoder[0].shape != (16000,1):
        return wav_decoder.audio
        
    
    # Randomly shift position to either 100ms forward or backward 

        
    position = random.randint(0,1)
    if position == 0:
        paddings = [[0, int(model_settings['desired_samples']*0.1)], [0, 0]]
        begin = [int(model_settings['desired_samples']*0.1),0]
    if position == 1:
        paddings = [[int(model_settings['desired_samples']*0.1), 0], [0, 0]]
        begin = [0,0]
    scaled_wav_decoder = tf.multiply(wav_decoder.audio, 1)
    padded_wav_decoder = tf.pad(scaled_wav_decoder, paddings)
    sliced_wav_decoder = tf.slice(padded_wav_decoder, begin = begin, size = [model_settings['desired_samples'], -1])

    # Take random silence file and add it to the signal in range between 0 and 0.1

    rand_silence = random.randint(0,len(silence_filenames) - 1)
    rand_amount = random.uniform(0, 0.1)
    silence_loader = tf.io.read_file(join(silence_dir,silence_filenames[rand_silence]))
    silence_decoder = tf.audio.decode_wav(silence_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    silence_padded_wav = tf.add(sliced_wav_decoder, tf.multiply(silence_decoder.audio, rand_amount))
    silence_padded_wav = tf.squeeze(silence_padded_wav)
    silence_padded_wav = silence_padded_wav.numpy()

    # Create MFCC of the augmented signal
    
    output = librosa.feature.mfcc(y=silence_padded_wav,sr=model_settings['desired_samples'],n_mfcc=model_settings['fingerprint_width'],n_fft=512,win_length=model_settings['window_size_samples'],hop_length=model_settings['window_stride_samples'])



    return output


In [27]:
# Function that creates the MFCC and check if they have the desired length 

def extract_features(samples, labels, num_ceps):
    prob_cnt = 0
    out_x = []
    out_y = []
        
    for index, filename in enumerate(samples):

        # Create path from given filename and target item
        path = join(data_dir, targets[int(labels[index])], 
                    filename)

        # Create MFCCs
        mfccs = calc_mfcc(path,num_ceps)

        if mfccs.shape[1] == num_ceps:
            out_x.append(mfccs)
            out_y.append(labels[index])
            print('Correctly computed :', index)
        else:
            print('Dropped:', index, mfccs.shape)
            prob_cnt += 1

    return out_x, out_y, prob_cnt

In [28]:
# Function for creating noise with the desired SNR
# According to https://medium.com/analytics-vidhya/adding-noise-to-audio-clips-5d8cee24ccb8

def calc_noise_for_SNR(signal,noise,SNR):
    
    #Calculate RMS from signal 
    RMS_s = (np.sqrt(np.mean(np.square(signal)))).astype('float64')
    
    #Calculate RMS from noise
    RMS_n_current = (np.sqrt(np.mean(np.square(noise)))).astype('float64')
    
    #Calculate required RMS from noise 
    RMS_n = (np.sqrt(np.square(RMS_s)/(np.power(10,SNR/10)))).astype('float64')
    
    noise = noise * (RMS_n/RMS_n_current)
    
    return noise

In [29]:
# Function: Create MFCC from given path with the desired SNR

def calc_mfcc_SNR(path,num_ceps,SNR):
    
    # Load wave file
    
    wav_loader = tf.io.read_file(path)
    wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    #wav_decoder = tf.audio.decode_wav(wav_loader,desired_channels=1, desired_samples=-1)
    if wav_decoder[0].shape != (16000,1):
        return wav_decoder.audio
        
    
    # Randomly shift position to either 100ms forward or backward  

        
    position = random.randint(0,1)
    if position == 0:
        paddings = [[0, int(model_settings['desired_samples']*0.1)], [0, 0]]
        begin = [int(model_settings['desired_samples']*0.1),0]
    if position == 1:
        paddings = [[int(model_settings['desired_samples']*0.1), 0], [0, 0]]
        begin = [0,0]
    scaled_wav_decoder = tf.multiply(wav_decoder.audio, 1)
    padded_wav_decoder = tf.pad(scaled_wav_decoder, paddings)
    sliced_wav_decoder = tf.slice(padded_wav_decoder, begin = begin, size = [model_settings['desired_samples'], -1])

     # Add random noise file with a certain ratio (SNR)

    rand_silence = random.randint(0,len(silence_filenames) - 1)
    rand_amount = random.uniform(0, 0.1)
    silence_loader = tf.io.read_file(join(silence_dir,silence_filenames[rand_silence]))
    silence_decoder = tf.audio.decode_wav(silence_loader,desired_channels=1, desired_samples=model_settings['desired_samples'])
    
    # Get the audio data from the decoder 
    # Data has to be float 64 in order to add small amounts of noise, e.g., SNR of 1000
    
    noise = silence_decoder.audio
    noise = np.array(noise,dtype='float64')
    
    # Shape noise in order to match a certain SNR
    
    noise = calc_noise_for_SNR(sliced_wav_decoder,noise,SNR)
    
    
    sliced_wav_decoder = np.array(sliced_wav_decoder,dtype='float64')
    silence_padded_wav = tf.add(sliced_wav_decoder, noise)
    
    # Add the desired noise and check if it worked
    
    power = np.square(sliced_wav_decoder)
    average_power = np.mean(power)
    average_power_db = 10 * np.log10(average_power)
    
    #print('DB: ', average_power_db)
    
    power_noise = np.square(noise)
    average_power_noise = np.mean(power_noise)
    average_power_db_noise = 10 * np.log10(average_power_noise)
    
    print('SNR : ', (average_power_db - average_power_db_noise))
    
    
    silence_padded_wav = tf.squeeze(silence_padded_wav)
    silence_padded_wav = silence_padded_wav.numpy()

    # Create MFCC of the augmented signal 

    output = librosa.feature.mfcc(y=silence_padded_wav,sr=model_settings['desired_samples'],n_mfcc=model_settings['fingerprint_width'],n_fft=512,win_length=model_settings['window_size_samples'],hop_length=model_settings['window_stride_samples'])



    return output

In [30]:
# Function that creates the MFCC and check if they have the desired length and a certain SNR

def extract_features_SNR(samples, labels, num_ceps, SNR):
    prob_cnt = 0
    out_x = []
    out_y = []
        
    for index, filename in enumerate(samples):

        # Create path from given filename and target item
        path = join(data_dir, targets[int(labels[index])], 
                    filename)

        # Create MFCCs
        mfccs = calc_mfcc_SNR(path,num_ceps,SNR)

        if mfccs.shape[1] == num_ceps:
            out_x.append(mfccs)
            out_y.append(labels[index])
            print('Correctly computed :', index)
        else:
            print('Dropped:', index, mfccs.shape)
            prob_cnt += 1

    return out_x, out_y, prob_cnt

<font size="5">3. Save the MFCCs </font>

Save the MFCCs including their targets seperated into training, validation and test set. Including the test sets with the different SNRs.

In [31]:
# Create test set with SNR of 1
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=1)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'all_test_SNR_1.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  0.9999997391696507
Correctly computed : 0
SNR :  1.0000007720793178
Correctly computed : 1
SNR :  1.0000003795181982
Correctly computed : 2
SNR :  1.0000002150766782
Correctly computed : 3
SNR :  0.999999671040186
Correctly computed : 4
SNR :  0.9999997802504996
Correctly computed : 5
SNR :  1.0000002977271407
Correctly computed : 6
SNR :  1.0000003245367566
Correctly computed : 7
SNR :  0.9999997813666361
Correctly computed : 8
SNR :  1.000000025526191
Correctly computed : 9
SNR :  0.9999998132529271
Correctly computed : 10
SNR :  0.9999998574479037
Correctly computed : 11
SNR :  1.0000000118434365
Correctly computed : 12
SNR :  1.0000006627796054
Correctly computed : 13
SNR :  1.0000002559020071
Correctly computed : 14
SNR :  1.0000001809973398
Correctly computed : 15
SNR :  0.9999996896366028
Correctly computed : 16
SNR :  0.9999997945254613
Correctly computed : 17
SNR :  0.999999965722477
Correctly computed : 18
SNR :  1.0000001775192793
Correctly computed : 19
SNR :  1.0000

Correctly computed : 168
SNR :  1.0000004270229184
Correctly computed : 169
SNR :  0.9999998924160494
Correctly computed : 170
SNR :  1.0000000646598224
Correctly computed : 171
SNR :  1.000000159782168
Correctly computed : 172
SNR :  1.0000006879952217
Correctly computed : 173
SNR :  0.9999999545193674
Correctly computed : 174
SNR :  1.0000002727131196
Correctly computed : 175
SNR :  1.0000003386294054
Correctly computed : 176
SNR :  1.0000000141686556
Correctly computed : 177
SNR :  1.0000002756405877
Correctly computed : 178
SNR :  1.000000194240485
Correctly computed : 179
SNR :  1.000000053335384
Correctly computed : 180
SNR :  0.9999997391012343
Correctly computed : 181
SNR :  1.0000004948657093
Correctly computed : 182
SNR :  1.0000002629144866
Correctly computed : 183
SNR :  1.0000002355121183
Correctly computed : 184
SNR :  1.000000009062905
Correctly computed : 185
SNR :  1.0000004615440758
Correctly computed : 186
SNR :  1.000000204927158
Correctly computed : 187
SNR :  1.00

Correctly computed : 344
SNR :  1.0000003540548228
Correctly computed : 345
SNR :  1.0000006185172623
Correctly computed : 346
SNR :  0.9999998693207779
Correctly computed : 347
SNR :  1.0000004988829332
Correctly computed : 348
SNR :  0.9999999726566529
Correctly computed : 349
SNR :  0.9999997266479355
Correctly computed : 350
SNR :  1.0000004403122382
Correctly computed : 351
SNR :  1.0000001975497739
Correctly computed : 352
SNR :  0.9999997435523724
Correctly computed : 353
SNR :  1.000000099710988
Correctly computed : 354
SNR :  1.0000001890348038
Correctly computed : 355
SNR :  0.9999999349275974
Correctly computed : 356
SNR :  0.9999995646087632
Correctly computed : 357
SNR :  0.9999998701081552
Correctly computed : 358
SNR :  0.9999997301568726
Correctly computed : 359
SNR :  0.999999783065185
Correctly computed : 360
SNR :  1.0000003361476608
Correctly computed : 361
SNR :  1.0000004100193038
Correctly computed : 362
SNR :  1.0000001486997263
Correctly computed : 363
SNR :  1

Correctly computed : 516
SNR :  1.0000006625005042
Correctly computed : 517
SNR :  0.9999992878068085
Correctly computed : 518
SNR :  1.0000005290552494
Correctly computed : 519
SNR :  0.9999999645065092
Correctly computed : 520
SNR :  1.0000001884016676
Correctly computed : 521
SNR :  1.0000000459643417
Correctly computed : 522
SNR :  1.0000004278934043
Correctly computed : 523
SNR :  0.999999836584589
Correctly computed : 524
SNR :  1.0000006792907357
Correctly computed : 525
SNR :  1.0000002804044072
Correctly computed : 526
SNR :  1.0000001282071693
Correctly computed : 527
SNR :  1.0000001123638
Correctly computed : 528
SNR :  1.0000002441397555
Correctly computed : 529
SNR :  1.000000499352442
Correctly computed : 530
SNR :  0.9999995262901962
Correctly computed : 531
SNR :  1.000000152160755
Correctly computed : 532
SNR :  0.9999998071840572
Correctly computed : 533
SNR :  1.0000001261985219
Correctly computed : 534
SNR :  0.9999995987814074
Correctly computed : 535
SNR :  0.999

SNR :  1.0000002749655685
Correctly computed : 691
SNR :  1.000000722562099
Correctly computed : 692
SNR :  1.0000000682459138
Correctly computed : 693
SNR :  1.0000002282371838
Correctly computed : 694
SNR :  0.9999997070601072
Correctly computed : 695
SNR :  1.0000000995470941
Correctly computed : 696
SNR :  1.0000003639126263
Correctly computed : 697
SNR :  0.9999995580411678
Correctly computed : 698
SNR :  0.9999998929867715
Correctly computed : 699
SNR :  0.9999997520366506
Correctly computed : 700
SNR :  0.9999998242320061
Correctly computed : 701
SNR :  1.0000000438832082
Correctly computed : 702
SNR :  0.9999997579573545
Correctly computed : 703
SNR :  0.9999999044393242
Correctly computed : 704
SNR :  0.9999999725942246
Correctly computed : 705
SNR :  1.0000003620244016
Correctly computed : 706
SNR :  0.9999996689964608
Correctly computed : 707
SNR :  0.9999998949562006
Correctly computed : 708
SNR :  1.0000002143700648
Correctly computed : 709
SNR :  1.0000004974153889
Correc

...

SNR :  0.9999997292601748
Correctly computed : 4019
SNR :  0.999999828990628
Correctly computed : 4020
SNR :  1.000000436755439
Correctly computed : 4021
SNR :  1.0000001779278662
Correctly computed : 4022
SNR :  0.9999997361738977
Correctly computed : 4023
SNR :  1.0000003667840787
Correctly computed : 4024
SNR :  1.0000003902340993
Correctly computed : 4025
SNR :  1.0000006219880468
Correctly computed : 4026
SNR :  1.0000005014976168
Correctly computed : 4027
SNR :  0.9999997448632989
Correctly computed : 4028
SNR :  1.0000004703733296
Correctly computed : 4029
SNR :  1.0000000121652235
Correctly computed : 4030
SNR :  0.9999997981945583
Correctly computed : 4031
SNR :  1.0000002182752112
Correctly computed : 4032
SNR :  1.0000010168195708
Correctly computed : 4033
SNR :  0.9999994118421824
Correctly computed : 4034
SNR :  1.0000002559020054
Correctly computed : 4035
SNR :  1.0000002259829905
Correctly computed : 4036
SNR :  1.0000003835006268
Correctly computed : 4037
SNR :  1.00000

SNR :  0.9999997077700726
Correctly computed : 4195
SNR :  1.000000425261856
Correctly computed : 4196
SNR :  1.0000001832142758
Correctly computed : 4197
SNR :  1.0000000799770064
Correctly computed : 4198
SNR :  1.0000002785087894
Correctly computed : 4199
SNR :  1.0000006239833787
Correctly computed : 4200
SNR :  1.000000166797129
Correctly computed : 4201
SNR :  1.0000006969989208
Correctly computed : 4202
SNR :  1.0000003465822687
Correctly computed : 4203
SNR :  1.0000005088303396
Correctly computed : 4204
SNR :  1.0000008069432411
Correctly computed : 4205
SNR :  0.9999999395421781
Correctly computed : 4206
SNR :  1.000000399518747
Correctly computed : 4207
SNR :  1.0000001746905909
Correctly computed : 4208
SNR :  1.0000004235603797
Correctly computed : 4209
SNR :  0.9999997790880286
Correctly computed : 4210
SNR :  0.9999993133170797
Correctly computed : 4211
SNR :  0.9999998983340816
Correctly computed : 4212
SNR :  0.9999997641529568
Correctly computed : 4213
SNR :  1.000000

SNR :  1.0000002947734075
Correctly computed : 4371
SNR :  1.0000006089585725
Correctly computed : 4372
SNR :  1.0000000081985618
Correctly computed : 4373
SNR :  0.9999994048517813
Correctly computed : 4374
SNR :  1.0000003510101756
Correctly computed : 4375
SNR :  1.0000003526626955
Correctly computed : 4376
SNR :  1.0000001489723545
Correctly computed : 4377
SNR :  0.9999999529655668
Correctly computed : 4378
SNR :  0.9999996818368047
Correctly computed : 4379
SNR :  0.9999993525112458
Correctly computed : 4380
SNR :  1.0000004720413358
Correctly computed : 4381
SNR :  1.0000000746016795
Correctly computed : 4382
SNR :  1.0000002318679329
Correctly computed : 4383
SNR :  1.0000002523809286
Correctly computed : 4384
SNR :  0.9999996564824158
Correctly computed : 4385
SNR :  1.000000416444582
Correctly computed : 4386
SNR :  1.000000259129532
Correctly computed : 4387
SNR :  1.0000005495332829
Correctly computed : 4388
SNR :  1.0000001208861953
Correctly computed : 4389
SNR :  0.99999

Correctly computed : 4548
SNR :  0.9999999811174725
Correctly computed : 4549
SNR :  1.0000000489093317
Correctly computed : 4550
SNR :  1.0000000829478068
Correctly computed : 4551
SNR :  1.0000003879516797
Correctly computed : 4552
SNR :  0.9999997857683667
Correctly computed : 4553
SNR :  0.9999996027743983
Correctly computed : 4554
SNR :  1.0000006167732707
Correctly computed : 4555
SNR :  0.9999999727935744
Correctly computed : 4556
SNR :  0.9999999813372185
Correctly computed : 4557
SNR :  0.9999993853757854
Correctly computed : 4558
SNR :  1.0000001239023817
Correctly computed : 4559
SNR :  1.000000747163476
Correctly computed : 4560
SNR :  0.9999997816561859
Correctly computed : 4561
SNR :  1.0000001953936675
Correctly computed : 4562
SNR :  1.0000001319049225
Correctly computed : 4563
SNR :  1.0000000202960848
Correctly computed : 4564
SNR :  0.9999999742619288
Correctly computed : 4565
SNR :  0.9999998955123957
Correctly computed : 4566
SNR :  0.9999995726223858
Correctly com

Correctly computed : 4724
SNR :  1.0000001796418765
Correctly computed : 4725
SNR :  1.0000008259384785
Correctly computed : 4726
SNR :  0.999999601334892
Correctly computed : 4727
SNR :  0.9999998701081552
Correctly computed : 4728
SNR :  0.9999996017909751
Correctly computed : 4729
SNR :  0.9999993880846105
Correctly computed : 4730
SNR :  1.0000004305670345
Correctly computed : 4731
SNR :  1.0000003939904687
Correctly computed : 4732
SNR :  1.0000003865630482
Correctly computed : 4733
SNR :  1.000000520172886
Correctly computed : 4734
SNR :  0.9999999493193279
Correctly computed : 4735
SNR :  1.000000545670602
Correctly computed : 4736
SNR :  0.999999571056378
Correctly computed : 4737
SNR :  0.9999999476520784
Correctly computed : 4738
SNR :  1.0000000819231758
Correctly computed : 4739
SNR :  1.0000001306449384
Correctly computed : 4740
SNR :  0.9999999077232289
Correctly computed : 4741
SNR :  1.0000004912316633
Correctly computed : 4742
SNR :  0.9999995966105821
Correctly comput

In [32]:
# Create test set with SNR of 10
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=10)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'all_test_SNR_10.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  10.00000024782303
Correctly computed : 0
SNR :  10.000000276817694
Correctly computed : 1
SNR :  10.000000379518195
Correctly computed : 2
SNR :  10.000000215076678
Correctly computed : 3
SNR :  10.000000113881537
Correctly computed : 4
SNR :  9.999999780250501
Correctly computed : 5
SNR :  9.999999720557746
Correctly computed : 6
SNR :  10.000000324536757
Correctly computed : 7
SNR :  9.999999818856057
Correctly computed : 8
SNR :  10.00000053430567
Correctly computed : 9
SNR :  10.000000217977764
Correctly computed : 10
SNR :  9.99999995042728
Correctly computed : 11
SNR :  9.999999763659826
Correctly computed : 12
SNR :  10.000000102872328
Correctly computed : 13
SNR :  10.000000499851133
Correctly computed : 14
SNR :  10.0000002440844
Correctly computed : 15
SNR :  9.999999689636603
Correctly computed : 16
SNR :  10.000000376756159
Correctly computed : 17
SNR :  9.999999965722477
Correctly computed : 18
SNR :  10.000000333217514
Correctly computed : 19
SNR :  10.000000242493

Correctly computed : 168
SNR :  10.000000357321937
Correctly computed : 169
SNR :  9.99999989241605
Correctly computed : 170
SNR :  10.000000064659819
Correctly computed : 171
SNR :  10.000000159782168
Correctly computed : 172
SNR :  9.99999974072928
Correctly computed : 173
SNR :  10.000000321350377
Correctly computed : 174
SNR :  10.00000027271312
Correctly computed : 175
SNR :  10.000000338629405
Correctly computed : 176
SNR :  9.999999849665052
Correctly computed : 177
SNR :  10.000000275640595
Correctly computed : 178
SNR :  10.000000431677186
Correctly computed : 179
SNR :  9.999999903645914
Correctly computed : 180
SNR :  9.999999739101241
Correctly computed : 181
SNR :  10.00000049486571
Correctly computed : 182
SNR :  10.000000289473732
Correctly computed : 183
SNR :  10.000000235512118
Correctly computed : 184
SNR :  10.000000009062909
Correctly computed : 185
SNR :  9.999999955540964
Correctly computed : 186
SNR :  10.000000244386626
Correctly computed : 187
SNR :  10.000000

Correctly computed : 338
SNR :  9.999999935710104
Correctly computed : 339
SNR :  10.000000242542257
Correctly computed : 340
SNR :  10.000000452937599
Correctly computed : 341
SNR :  10.000000139263854
Correctly computed : 342
SNR :  10.000000027066687
Correctly computed : 343
SNR :  9.999999652886167
Correctly computed : 344
SNR :  10.00000035405482
Correctly computed : 345
SNR :  10.000000312753386
Correctly computed : 346
SNR :  9.999999734840689
Correctly computed : 347
SNR :  10.000000670766315
Correctly computed : 348
SNR :  9.999999972656653
Correctly computed : 349
SNR :  10.00000124061372
Correctly computed : 350
SNR :  10.000000044011571
Correctly computed : 351
SNR :  10.00000019754977
Correctly computed : 352
SNR :  9.999999743552365
Correctly computed : 353
SNR :  9.999999814111753
Correctly computed : 354
SNR :  10.000000310038743
Correctly computed : 355
SNR :  10.00000021800112
Correctly computed : 356
SNR :  9.99999956460876
Correctly computed : 357
SNR :  9.999999866

SNR :  10.000000322634158
Correctly computed : 513
SNR :  10.000000056746288
Correctly computed : 514
SNR :  10.000000399618209
Correctly computed : 515
SNR :  10.000000576991418
Correctly computed : 516
SNR :  10.00000024711768
Correctly computed : 517
SNR :  9.999999738327503
Correctly computed : 518
SNR :  10.000000618769615
Correctly computed : 519
SNR :  9.99999996450651
Correctly computed : 520
SNR :  10.000000188401668
Correctly computed : 521
SNR :  9.99999994932518
Correctly computed : 522
SNR :  10.000000427893404
Correctly computed : 523
SNR :  9.999999836584589
Correctly computed : 524
SNR :  10.000000679290736
Correctly computed : 525
SNR :  10.000000280404407
Correctly computed : 526
SNR :  10.000000132020197
Correctly computed : 527
SNR :  10.0000001123638
Correctly computed : 528
SNR :  10.00000030348776
Correctly computed : 529
SNR :  10.000000317939215
Correctly computed : 530
SNR :  9.999999671103112
Correctly computed : 531
SNR :  10.000000152160755
Correctly comput

Correctly computed : 684
SNR :  10.00000040530762
Correctly computed : 685
SNR :  9.99999969705079
Correctly computed : 686
SNR :  10.000000268516423
Correctly computed : 687
SNR :  9.999999549714397
Correctly computed : 688
SNR :  10.000000096590917
Correctly computed : 689
SNR :  10.000000623263332
Correctly computed : 690
SNR :  10.000000461025152
Correctly computed : 691
SNR :  10.000000722562092
Correctly computed : 692
SNR :  10.00000006824591
Correctly computed : 693
SNR :  9.999999857666598
Correctly computed : 694
SNR :  9.99999978511844
Correctly computed : 695
SNR :  10.000000099547094
Correctly computed : 696
SNR :  10.000000165791747
Correctly computed : 697
SNR :  9.999999558041168
Correctly computed : 698
SNR :  10.000000059574447
Correctly computed : 699
SNR :  10.00000081654914
Correctly computed : 700
SNR :  10.000000160440173
Correctly computed : 701
SNR :  9.999999786837439
Correctly computed : 702
SNR :  9.999999757957355
Correctly computed : 703
SNR :  9.999999904

...

SNR :  9.999999438707484
Correctly computed : 3962
SNR :  9.999999980450518
Correctly computed : 3963
SNR :  10.000000640244966
Correctly computed : 3964
SNR :  9.999999939863635
Correctly computed : 3965
SNR :  9.999999974163906
Correctly computed : 3966
SNR :  9.999999502519056
Correctly computed : 3967
SNR :  10.000000378632368
Correctly computed : 3968
SNR :  9.999999523174523
Correctly computed : 3969
SNR :  9.999999444764612
Correctly computed : 3970
SNR :  10.000000013682367
Correctly computed : 3971
SNR :  9.99999964234366
Correctly computed : 3972
SNR :  10.000000143521806
Correctly computed : 3973
SNR :  10.000000930378107
Correctly computed : 3974
SNR :  9.999999838546668
Correctly computed : 3975
SNR :  10.000000105998245
Correctly computed : 3976
SNR :  9.999999695137497
Correctly computed : 3977
SNR :  10.000000663965608
Correctly computed : 3978
SNR :  10.000000541253634
Correctly computed : 3979
SNR :  9.999999876001553
Correctly computed : 3980
SNR :  9.999999816322262

Correctly computed : 4134
SNR :  10.00000060289986
Correctly computed : 4135
SNR :  10.000000498879615
Correctly computed : 4136
SNR :  9.9999996324538
Correctly computed : 4137
SNR :  10.000000144426707
Correctly computed : 4138
SNR :  10.000000413385425
Correctly computed : 4139
SNR :  9.999999715585894
Correctly computed : 4140
SNR :  10.00000006479653
Correctly computed : 4141
SNR :  10.000000405635848
Correctly computed : 4142
SNR :  9.9999996987722
Correctly computed : 4143
SNR :  9.999999659213195
Correctly computed : 4144
SNR :  9.999999604371673
Correctly computed : 4145
SNR :  10.000000677760312
Correctly computed : 4146
SNR :  9.999999901834151
Correctly computed : 4147
SNR :  10.000000113873398
Correctly computed : 4148
SNR :  10.00000004965122
Correctly computed : 4149
SNR :  10.000001116793378
Correctly computed : 4150
SNR :  10.000000379555477
Correctly computed : 4151
SNR :  9.999999890928216
Correctly computed : 4152
SNR :  9.999999887392626
Correctly computed : 4153
S

SNR :  10.000000318604016
Correctly computed : 4310
SNR :  10.000000202985525
Correctly computed : 4311
SNR :  10.000000221207465
Correctly computed : 4312
SNR :  10.000000839712207
Correctly computed : 4313
SNR :  10.000000348229094
Correctly computed : 4314
SNR :  9.999999648850078
Correctly computed : 4315
SNR :  10.00000036081277
Correctly computed : 4316
SNR :  10.000000751053701
Correctly computed : 4317
SNR :  10.000000533857751
Correctly computed : 4318
SNR :  9.999999735708563
Correctly computed : 4319
SNR :  10.000000758664381
Correctly computed : 4320
SNR :  9.999999669405884
Correctly computed : 4321
SNR :  9.999999816687893
Correctly computed : 4322
SNR :  10.000000403258163
Correctly computed : 4323
SNR :  9.999999729660928
Correctly computed : 4324
SNR :  9.99999931470878
Correctly computed : 4325
SNR :  10.000000150074868
Correctly computed : 4326
SNR :  10.000000020995074
Correctly computed : 4327
SNR :  9.999999417019747
Correctly computed : 4328
SNR :  10.00000063979

Correctly computed : 4484
SNR :  9.99999983200842
Correctly computed : 4485
SNR :  10.000000066054252
Correctly computed : 4486
SNR :  10.000000399937761
Correctly computed : 4487
SNR :  10.000000287434737
Correctly computed : 4488
SNR :  10.000000108737886
Correctly computed : 4489
SNR :  10.00000075618043
Correctly computed : 4490
SNR :  10.000000222622361
Correctly computed : 4491
SNR :  10.00000046287699
Correctly computed : 4492
SNR :  10.000000156389632
Correctly computed : 4493
SNR :  10.000000448213228
Correctly computed : 4494
SNR :  9.999999651160739
Correctly computed : 4495
SNR :  10.000000070387486
Correctly computed : 4496
SNR :  9.999999988410782
Correctly computed : 4497
SNR :  9.999999845393859
Correctly computed : 4498
SNR :  9.999999767699109
Correctly computed : 4499
SNR :  10.000000057399646
Correctly computed : 4500
SNR :  9.999999566645354
Correctly computed : 4501
SNR :  10.000000312172556
Correctly computed : 4502
SNR :  9.999999793048126
Correctly computed : 4

Correctly computed : 4659
SNR :  9.999999921644626
Correctly computed : 4660
SNR :  10.000000559148948
Correctly computed : 4661
SNR :  10.000000147288652
Correctly computed : 4662
SNR :  10.000000280361618
Correctly computed : 4663
SNR :  9.999999500787986
Correctly computed : 4664
SNR :  10.000000193534419
Correctly computed : 4665
SNR :  10.00000005798234
Correctly computed : 4666
SNR :  9.999999889431418
Correctly computed : 4667
SNR :  10.000000103692305
Correctly computed : 4668
SNR :  9.999999810187049
Correctly computed : 4669
SNR :  10.000000291220832
Correctly computed : 4670
SNR :  10.000000817232259
Correctly computed : 4671
SNR :  9.99999970013101
Correctly computed : 4672
SNR :  10.000000225846506
Correctly computed : 4673
SNR :  10.000000282112232
Correctly computed : 4674
SNR :  10.000000396745012
Correctly computed : 4675
SNR :  9.999999921121631
Correctly computed : 4676
SNR :  10.000000714035682
Correctly computed : 4677
SNR :  10.000000539453957
Correctly computed :

In [33]:
# Create test set with SNR of 20
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=20)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'all_test_SNR_20.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  20.00000024782304
Correctly computed : 0
SNR :  20.000000772079318
Correctly computed : 1
SNR :  20.000000379518195
Correctly computed : 2
SNR :  20.00000051885759
Correctly computed : 3
SNR :  20.000000113881537
Correctly computed : 4
SNR :  20.00000008274327
Correctly computed : 5
SNR :  20.00000029772714
Correctly computed : 6
SNR :  20.000000324536757
Correctly computed : 7
SNR :  19.99999978136664
Correctly computed : 8
SNR :  20.000000534305663
Correctly computed : 9
SNR :  19.99999981325292
Correctly computed : 10
SNR :  19.999999857447904
Correctly computed : 11
SNR :  19.99999976365982
Correctly computed : 12
SNR :  20.000000102872335
Correctly computed : 13
SNR :  20.00000025590201
Correctly computed : 14
SNR :  20.0000002440844
Correctly computed : 15
SNR :  20.000000274783464
Correctly computed : 16
SNR :  20.000000376756155
Correctly computed : 17
SNR :  20.00000067705114
Correctly computed : 18
SNR :  20.00000017751928
Correctly computed : 19
SNR :  20.000000242493

Correctly computed : 175
SNR :  20.000000743470604
Correctly computed : 176
SNR :  20.000000014168652
Correctly computed : 177
SNR :  20.000000275640595
Correctly computed : 178
SNR :  20.000000431677186
Correctly computed : 179
SNR :  20.00000005333538
Correctly computed : 180
SNR :  20.000000851927382
Correctly computed : 181
SNR :  20.00000049486571
Correctly computed : 182
SNR :  20.000000289473732
Correctly computed : 183
SNR :  20.00000023551212
Correctly computed : 184
SNR :  20.00000024035367
Correctly computed : 185
SNR :  19.999999955540964
Correctly computed : 186
SNR :  20.000000244386634
Correctly computed : 187
SNR :  20.000000663157145
Correctly computed : 188
SNR :  19.99999982544896
Correctly computed : 189
SNR :  20.00000019568608
Correctly computed : 190
SNR :  20.00000005612565
Correctly computed : 191
SNR :  19.999999996906954
Correctly computed : 192
SNR :  19.99999949531508
Correctly computed : 193
SNR :  20.000000422057173
Correctly computed : 194
SNR :  20.0000

Correctly computed : 356
SNR :  19.99999956460876
Correctly computed : 357
SNR :  19.99999986648165
Correctly computed : 358
SNR :  20.000000152027287
Correctly computed : 359
SNR :  20.000000825548145
Correctly computed : 360
SNR :  20.000000018098266
Correctly computed : 361
SNR :  20.000000228226863
Correctly computed : 362
SNR :  20.00000014869972
Correctly computed : 363
SNR :  20.000000389647553
Correctly computed : 364
SNR :  19.999999795110966
Correctly computed : 365
SNR :  20.00000065599988
Correctly computed : 366
SNR :  20.000000329784463
Correctly computed : 367
SNR :  19.99999971742512
Correctly computed : 368
SNR :  20.000000078797616
Correctly computed : 369
SNR :  19.99999942137475
Correctly computed : 370
SNR :  19.99999992014164
Correctly computed : 371
SNR :  19.99999997912756
Correctly computed : 372
SNR :  20.000000289434325
Correctly computed : 373
SNR :  20.00000020612449
Correctly computed : 374
SNR :  20.000000889536047
Correctly computed : 375
SNR :  19.99999

SNR :  20.000000126198522
Correctly computed : 534
SNR :  20.000000009926737
Correctly computed : 535
SNR :  19.99999996043779
Correctly computed : 536
SNR :  19.99999993428412
Correctly computed : 537
SNR :  20.000000351275602
Correctly computed : 538
SNR :  20.00000011302298
Correctly computed : 539
SNR :  20.00000069047495
Correctly computed : 540
SNR :  20.00000019912442
Correctly computed : 541
SNR :  20.00000061074929
Correctly computed : 542
SNR :  20.00000057748087
Correctly computed : 543
SNR :  20.000000469952617
Correctly computed : 544
SNR :  20.000000504985127
Correctly computed : 545
SNR :  20.000000096419193
Correctly computed : 546
SNR :  20.000000249548307
Correctly computed : 547
SNR :  20.000000210730086
Correctly computed : 548
SNR :  20.000000467763595
Correctly computed : 549
SNR :  20.000000026901283
Correctly computed : 550
SNR :  19.99999991318527
Correctly computed : 551
SNR :  20.00000028043737
Correctly computed : 552
SNR :  20.000000392557528
Correctly comp

Correctly computed : 709
SNR :  20.00000049741539
Correctly computed : 710
SNR :  20.00000009041186
Correctly computed : 711
SNR :  20.000000334341372
Correctly computed : 712
SNR :  19.999999652059408
Correctly computed : 713
SNR :  20.000000382402867
Correctly computed : 714
SNR :  20.0000001476934
Correctly computed : 715
SNR :  19.999999271352223
Correctly computed : 716
SNR :  20.000000101264394
Correctly computed : 717
SNR :  19.999999637782516
Correctly computed : 718
SNR :  19.999999247459243
Correctly computed : 719
SNR :  19.999999565449663
Correctly computed : 720
SNR :  19.99999962474075
Correctly computed : 721
SNR :  20.000000269221083
Correctly computed : 722
SNR :  19.999999517898512
Correctly computed : 723
SNR :  20.00000000402361
Correctly computed : 724
SNR :  20.000000072634613
Correctly computed : 725
SNR :  20.000000249364305
Correctly computed : 726
SNR :  19.999999948065827
Correctly computed : 727
SNR :  19.999999102270213
Correctly computed : 728
SNR :  20.00

...

SNR :  19.999999890444972
Correctly computed : 4073
SNR :  20.000000431110415
Correctly computed : 4074
SNR :  20.000000221517983
Correctly computed : 4075
SNR :  19.999999888929633
Correctly computed : 4076
SNR :  20.000000119136796
Correctly computed : 4077
SNR :  19.999999869089443
Correctly computed : 4078
SNR :  20.000000429829146
Correctly computed : 4079
SNR :  20.000000391474643
Correctly computed : 4080
SNR :  19.99999989443094
Correctly computed : 4081
SNR :  20.000000311785158
Correctly computed : 4082
SNR :  20.000000285244653
Correctly computed : 4083
SNR :  20.000000146384917
Correctly computed : 4084
SNR :  19.99999950997659
Correctly computed : 4085
SNR :  20.000000027354705
Correctly computed : 4086
SNR :  20.000000346812737
Correctly computed : 4087
SNR :  20.00000036495937
Correctly computed : 4088
SNR :  20.00000014245466
Correctly computed : 4089
SNR :  20.000000586659375
Correctly computed : 4090
SNR :  20.00000092261999
Correctly computed : 4091
SNR :  20.0000004

Correctly computed : 4250
SNR :  20.000000008029286
Correctly computed : 4251
SNR :  20.00000028255746
Correctly computed : 4252
SNR :  19.999999870097035
Correctly computed : 4253
SNR :  19.999999947503362
Correctly computed : 4254
SNR :  20.000000457578928
Correctly computed : 4255
SNR :  19.99999985258888
Correctly computed : 4256
SNR :  19.999999413699598
Correctly computed : 4257
SNR :  19.999999855458544
Correctly computed : 4258
SNR :  20.000000249713786
Correctly computed : 4259
SNR :  19.99999976342278
Correctly computed : 4260
SNR :  19.999999762925672
Correctly computed : 4261
SNR :  20.00000057748087
Correctly computed : 4262
SNR :  19.99999979168161
Correctly computed : 4263
SNR :  20.000000061948008
Correctly computed : 4264
SNR :  20.000000012401326
Correctly computed : 4265
SNR :  20.000000740801
Correctly computed : 4266
SNR :  20.000000153106342
Correctly computed : 4267
SNR :  20.000000384104496
Correctly computed : 4268
SNR :  20.000000791312356
Correctly computed :

SNR :  19.999999671972738
Correctly computed : 4426
SNR :  20.000000135770748
Correctly computed : 4427
SNR :  19.999999303725183
Correctly computed : 4428
SNR :  20.000000131892993
Correctly computed : 4429
SNR :  19.99999980022085
Correctly computed : 4430
SNR :  19.999999420007143
Correctly computed : 4431
SNR :  19.999999717522556
Correctly computed : 4432
SNR :  19.999999373696976
Correctly computed : 4433
SNR :  20.000000709739762
Correctly computed : 4434
SNR :  19.999999713094624
Correctly computed : 4435
SNR :  19.999999536553688
Correctly computed : 4436
SNR :  19.999999273708347
Correctly computed : 4437
SNR :  20.000000412622608
Correctly computed : 4438
SNR :  19.99999962805331
Correctly computed : 4439
SNR :  20.000000432330687
Correctly computed : 4440
SNR :  20.000000094233297
Correctly computed : 4441
SNR :  19.99999999335418
Correctly computed : 4442
SNR :  20.000000171049667
Correctly computed : 4443
SNR :  19.999999988848714
Correctly computed : 4444
SNR :  20.00000

Correctly computed : 4600
SNR :  19.9999997983915
Correctly computed : 4601
SNR :  20.000000281320645
Correctly computed : 4602
SNR :  19.99999954516382
Correctly computed : 4603
SNR :  20.000000232829198
Correctly computed : 4604
SNR :  20.000000763839097
Correctly computed : 4605
SNR :  19.999999566330764
Correctly computed : 4606
SNR :  19.999999886667496
Correctly computed : 4607
SNR :  19.99999992220453
Correctly computed : 4608
SNR :  19.999999997042753
Correctly computed : 4609
SNR :  19.99999995991472
Correctly computed : 4610
SNR :  19.99999965907098
Correctly computed : 4611
SNR :  20.000000416840564
Correctly computed : 4612
SNR :  19.99999997739892
Correctly computed : 4613
SNR :  20.00000045347219
Correctly computed : 4614
SNR :  20.000000433568445
Correctly computed : 4615
SNR :  19.99999969298686
Correctly computed : 4616
SNR :  19.99999987390325
Correctly computed : 4617
SNR :  19.999999892657378
Correctly computed : 4618
SNR :  20.00000062052129
Correctly computed : 46

SNR :  20.00000010784978
Correctly computed : 4779
SNR :  20.000000387974694
Correctly computed : 4780
SNR :  20.00000036697466
Correctly computed : 4781
SNR :  19.999999743402608
Correctly computed : 4782
SNR :  19.99999996769207
Correctly computed : 4783
SNR :  20.000000669524688
Correctly computed : 4784
SNR :  19.999999871020464
Correctly computed : 4785
SNR :  20.000000125510518
Correctly computed : 4786
SNR :  19.99999968669064
Correctly computed : 4787
SNR :  20.000000138680733
Correctly computed : 4788
SNR :  20.000000124938353
Correctly computed : 4789
SNR :  20.000000386655884
Correctly computed : 4790
SNR :  20.000000048706063
Correctly computed : 4791
SNR :  19.999999587497896
Correctly computed : 4792
SNR :  20.000000409139318
Correctly computed : 4793
SNR :  20.00000005600529
Correctly computed : 4794
SNR :  20.0000005523484
Correctly computed : 4795
SNR :  19.99999970050656
Correctly computed : 4796
SNR :  20.000000524110284
Correctly computed : 4797
SNR :  19.9999999722

In [34]:
# Create test set with SNR of 100
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=100)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'all_test_SNR_100.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  100.00000024782304
Correctly computed : 0
SNR :  100.00000077207933
Correctly computed : 1
SNR :  100.0000003795182
Correctly computed : 2
SNR :  100.00000051885759
Correctly computed : 3
SNR :  100.00000011388153
Correctly computed : 4
SNR :  99.9999997802505
Correctly computed : 5
SNR :  99.99999972055772
Correctly computed : 6
SNR :  100.00000097255581
Correctly computed : 7
SNR :  99.99999981885605
Correctly computed : 8
SNR :  100.00000002552618
Correctly computed : 9
SNR :  100.00000021797777
Correctly computed : 10
SNR :  99.9999998574479
Correctly computed : 11
SNR :  100.00000001184344
Correctly computed : 12
SNR :  100.00000010287232
Correctly computed : 13
SNR :  100.00000025590201
Correctly computed : 14
SNR :  100.00000018099733
Correctly computed : 15
SNR :  100.00000027478347
Correctly computed : 16
SNR :  99.99999979452546
Correctly computed : 17
SNR :  99.99999996572248
Correctly computed : 18
SNR :  100.00000017751928
Correctly computed : 19
SNR :  100.00000011

Correctly computed : 167
SNR :  100.00000038689328
Correctly computed : 168
SNR :  100.00000042702291
Correctly computed : 169
SNR :  99.9999998953626
Correctly computed : 170
SNR :  100.00000006465983
Correctly computed : 171
SNR :  99.99999984537561
Correctly computed : 172
SNR :  100.00000068799523
Correctly computed : 173
SNR :  99.99999995451937
Correctly computed : 174
SNR :  100.00000027271314
Correctly computed : 175
SNR :  100.0000007434706
Correctly computed : 176
SNR :  99.99999984966504
Correctly computed : 177
SNR :  100.00000058781336
Correctly computed : 178
SNR :  100.00000019424049
Correctly computed : 179
SNR :  99.99999990364591
Correctly computed : 180
SNR :  100.00000085192738
Correctly computed : 181
SNR :  100.0000004948657
Correctly computed : 182
SNR :  100.00000026291448
Correctly computed : 183
SNR :  100.00000023551212
Correctly computed : 184
SNR :  100.00000024035367
Correctly computed : 185
SNR :  99.99999995554096
Correctly computed : 186
SNR :  100.0000

Correctly computed : 336
SNR :  99.99999990090913
Correctly computed : 337
SNR :  99.99999985349915
Correctly computed : 338
SNR :  99.99999993571011
Correctly computed : 339
SNR :  99.99999995159743
Correctly computed : 340
SNR :  100.00000045293758
Correctly computed : 341
SNR :  100.00000013815757
Correctly computed : 342
SNR :  99.9999999680337
Correctly computed : 343
SNR :  99.99999965288617
Correctly computed : 344
SNR :  100.00000011007535
Correctly computed : 345
SNR :  100.00000031275339
Correctly computed : 346
SNR :  99.99999986932079
Correctly computed : 347
SNR :  100.00000067076633
Correctly computed : 348
SNR :  99.99999997265665
Correctly computed : 349
SNR :  99.99999972664793
Correctly computed : 350
SNR :  100.00000044031223
Correctly computed : 351
SNR :  100.00000019754978
Correctly computed : 352
SNR :  99.99999972825604
Correctly computed : 353
SNR :  100.00000009971099
Correctly computed : 354
SNR :  100.00000018903481
Correctly computed : 355
SNR :  100.000000

SNR :  99.99999927700331
Correctly computed : 507
SNR :  99.99999955078755
Correctly computed : 508
SNR :  99.99999976413197
Correctly computed : 509
SNR :  99.99999979989596
Correctly computed : 510
SNR :  100.0000002170283
Correctly computed : 511
SNR :  100.00000018492972
Correctly computed : 512
SNR :  100.00000036495453
Correctly computed : 513
SNR :  100.00000000666907
Correctly computed : 514
SNR :  100.00000034015508
Correctly computed : 515
SNR :  100.00000063724347
Correctly computed : 516
SNR :  100.00000024711768
Correctly computed : 517
SNR :  99.99999928780682
Correctly computed : 518
SNR :  100.00000052905524
Correctly computed : 519
SNR :  99.99999996450651
Correctly computed : 520
SNR :  100.00000018840169
Correctly computed : 521
SNR :  99.99999994932517
Correctly computed : 522
SNR :  100.0000000069648
Correctly computed : 523
SNR :  99.99999983658458
Correctly computed : 524
SNR :  100.00000008527914
Correctly computed : 525
SNR :  100.00000010687803
Correctly compu

SNR :  100.0000001008107
Correctly computed : 676
SNR :  100.00000032884795
Correctly computed : 677
SNR :  100.00000041770492
Correctly computed : 678
SNR :  99.9999996807806
Correctly computed : 679
SNR :  99.99999974099774
Correctly computed : 680
SNR :  100.00000061017207
Correctly computed : 681
SNR :  100.00000005962602
Correctly computed : 682
SNR :  99.99999983397092
Correctly computed : 683
SNR :  100.00000037124744
Correctly computed : 684
SNR :  100.0000000771555
Correctly computed : 685
SNR :  99.99999969705078
Correctly computed : 686
SNR :  100.00000030003778
Correctly computed : 687
SNR :  100.00000010291762
Correctly computed : 688
SNR :  100.00000009659092
Correctly computed : 689
SNR :  100.00000062326335
Correctly computed : 690
SNR :  100.00000027496557
Correctly computed : 691
SNR :  100.0000007225621
Correctly computed : 692
SNR :  99.99999986910416
Correctly computed : 693
SNR :  100.0000002282372
Correctly computed : 694
SNR :  99.99999970706011
Correctly comput

...

SNR :  100.0000006068849
Correctly computed : 4001
SNR :  100.00000016851942
Correctly computed : 4002
SNR :  99.9999997991702
Correctly computed : 4003
SNR :  100.00000058780702
Correctly computed : 4004
SNR :  100.00000009830632
Correctly computed : 4005
SNR :  100.00000026938096
Correctly computed : 4006
SNR :  100.00000036597649
Correctly computed : 4007
SNR :  100.00000009820127
Correctly computed : 4008
SNR :  99.9999993341841
Correctly computed : 4009
SNR :  99.9999999397646
Correctly computed : 4010
SNR :  100.00000030035739
Correctly computed : 4011
SNR :  100.00000002687793
Correctly computed : 4012
SNR :  100.00000028562597
Correctly computed : 4013
SNR :  100.00000025008826
Correctly computed : 4014
SNR :  100.00000019736876
Correctly computed : 4015
SNR :  100.00000036147414
Correctly computed : 4016
SNR :  100.00000036947628
Correctly computed : 4017
SNR :  100.00000034921968
Correctly computed : 4018
SNR :  99.99999972926018
Correctly computed : 4019
SNR :  100.000000705

Correctly computed : 4174
SNR :  99.99999983190474
Correctly computed : 4175
SNR :  100.00000091734844
Correctly computed : 4176
SNR :  99.9999997802505
Correctly computed : 4177
SNR :  99.99999977517386
Correctly computed : 4178
SNR :  99.99999998259753
Correctly computed : 4179
SNR :  99.99999973105926
Correctly computed : 4180
SNR :  100.00000022063222
Correctly computed : 4181
SNR :  99.99999997007693
Correctly computed : 4182
SNR :  100.00000091536727
Correctly computed : 4183
SNR :  99.99999987579879
Correctly computed : 4184
SNR :  99.99999928065786
Correctly computed : 4185
SNR :  100.0000003031791
Correctly computed : 4186
SNR :  99.999999718957
Correctly computed : 4187
SNR :  99.99999979925899
Correctly computed : 4188
SNR :  100.00000032195737
Correctly computed : 4189
SNR :  100.00000051857253
Correctly computed : 4190
SNR :  100.00000036780743
Correctly computed : 4191
SNR :  100.00000086661255
Correctly computed : 4192
SNR :  100.00000022411109
Correctly computed : 4193


Correctly computed : 4347
SNR :  100.00000034321457
Correctly computed : 4348
SNR :  100.0000001326517
Correctly computed : 4349
SNR :  100.00000039629226
Correctly computed : 4350
SNR :  99.99999945192366
Correctly computed : 4351
SNR :  100.00000048619023
Correctly computed : 4352
SNR :  99.99999983686862
Correctly computed : 4353
SNR :  99.9999998812462
Correctly computed : 4354
SNR :  100.00000071241885
Correctly computed : 4355
SNR :  99.99999999246901
Correctly computed : 4356
SNR :  100.00000043254619
Correctly computed : 4357
SNR :  100.00000032148913
Correctly computed : 4358
SNR :  100.00000030337353
Correctly computed : 4359
SNR :  100.00000007184823
Correctly computed : 4360
SNR :  100.00000036723893
Correctly computed : 4361
SNR :  100.0000001898964
Correctly computed : 4362
SNR :  100.00000024419062
Correctly computed : 4363
SNR :  99.99999960652885
Correctly computed : 4364
SNR :  99.9999993510851
Correctly computed : 4365
SNR :  99.99999998553764
Correctly computed : 43

Correctly computed : 4521
SNR :  100.00000003721925
Correctly computed : 4522
SNR :  99.99999938425634
Correctly computed : 4523
SNR :  100.00000028480741
Correctly computed : 4524
SNR :  99.99999982117717
Correctly computed : 4525
SNR :  100.00000018344275
Correctly computed : 4526
SNR :  99.99999988672937
Correctly computed : 4527
SNR :  99.99999951148976
Correctly computed : 4528
SNR :  100.00000050036097
Correctly computed : 4529
SNR :  100.00000009731673
Correctly computed : 4530
SNR :  100.00000006040634
Correctly computed : 4531
SNR :  99.99999959467013
Correctly computed : 4532
SNR :  99.99999961374789
Correctly computed : 4533
SNR :  100.00000002902013
Correctly computed : 4534
SNR :  99.99999958829989
Correctly computed : 4535
SNR :  99.99999903418377
Correctly computed : 4536
SNR :  99.99999987557877
Correctly computed : 4537
SNR :  99.99999951171495
Correctly computed : 4538
SNR :  100.00000023309366
Correctly computed : 4539
SNR :  100.00000010144241
Correctly computed : 4

SNR :  99.99999959394586
Correctly computed : 4697
SNR :  99.99999964767079
Correctly computed : 4698
SNR :  100.00000016341403
Correctly computed : 4699
SNR :  100.00000020772539
Correctly computed : 4700
SNR :  99.99999954510308
Correctly computed : 4701
SNR :  100.00000032158894
Correctly computed : 4702
SNR :  99.99999995144796
Correctly computed : 4703
SNR :  99.99999989869357
Correctly computed : 4704
SNR :  100.00000030649045
Correctly computed : 4705
SNR :  100.00000036524824
Correctly computed : 4706
SNR :  100.00000032744302
Correctly computed : 4707
SNR :  99.9999999089049
Correctly computed : 4708
SNR :  99.99999970411935
Correctly computed : 4709
SNR :  100.00000008093085
Correctly computed : 4710
SNR :  100.00000032370485
Correctly computed : 4711
SNR :  100.00000012377035
Correctly computed : 4712
SNR :  99.99999976540371
Correctly computed : 4713
SNR :  100.00000015243279
Correctly computed : 4714
SNR :  100.00000006988412
Correctly computed : 4715
SNR :  100.0000000471

In [35]:
# Create test set with SNR of 1000
# num_ceps is the desired length of the MFCC

num_ceps = 101
x_test, y_test, prob = extract_features_SNR(test_files, y_test, num_ceps, SNR=1000)
print('Removed percentage:', prob / len(y_test))

# Save MFCCs and their target

feature_sets_file = 'all_test_SNR_1000.npz'
np.savez(feature_sets_file, 
         x_test=x_test, 
         y_test=y_test)

SNR :  1000.000000247823
Correctly computed : 0
SNR :  1000.0000002768177
Correctly computed : 1
SNR :  1000.0000003795184
Correctly computed : 2
SNR :  1000.0000002150769
Correctly computed : 3
SNR :  1000.0000001138815
Correctly computed : 4
SNR :  1000.0000000827433
Correctly computed : 5
SNR :  1000.0000002977273
Correctly computed : 6
SNR :  1000.0000009725559
Correctly computed : 7
SNR :  999.9999997813667
Correctly computed : 8
SNR :  1000.0000000255262
Correctly computed : 9
SNR :  999.999999813253
Correctly computed : 10
SNR :  999.999999857448
Correctly computed : 11
SNR :  999.9999997636598
Correctly computed : 12
SNR :  1000.0000001028724
Correctly computed : 13
SNR :  1000.0000002559019
Correctly computed : 14
SNR :  1000.0000002440844
Correctly computed : 15
SNR :  999.9999996896365
Correctly computed : 16
SNR :  1000.0000003767562
Correctly computed : 17
SNR :  1000.0000006770513
Correctly computed : 18
SNR :  1000.0000001775194
Correctly computed : 19
SNR :  1000.000000

Correctly computed : 168
SNR :  1000.000000357322
Correctly computed : 169
SNR :  999.999999892416
Correctly computed : 170
SNR :  1000.0000000646597
Correctly computed : 171
SNR :  999.9999998453757
Correctly computed : 172
SNR :  1000.0000006879952
Correctly computed : 173
SNR :  1000.0000003213505
Correctly computed : 174
SNR :  999.9999998581783
Correctly computed : 175
SNR :  1000.0000003386293
Correctly computed : 176
SNR :  999.999999849665
Correctly computed : 177
SNR :  1000.0000005878134
Correctly computed : 178
SNR :  1000.0000001942404
Correctly computed : 179
SNR :  999.9999999036459
Correctly computed : 180
SNR :  999.9999997391012
Correctly computed : 181
SNR :  1000.0000004948656
Correctly computed : 182
SNR :  1000.0000002629145
Correctly computed : 183
SNR :  1000.0000002355122
Correctly computed : 184
SNR :  1000.0000002403536
Correctly computed : 185
SNR :  999.9999999555409
Correctly computed : 186
SNR :  1000.0000002049272
Correctly computed : 187
SNR :  1000.0000

Correctly computed : 332
SNR :  999.999999930258
Correctly computed : 333
SNR :  1000.0000006964883
Correctly computed : 334
SNR :  999.9999998795652
Correctly computed : 335
SNR :  1000.0000006722978
Correctly computed : 336
SNR :  999.9999999009093
Correctly computed : 337
SNR :  1000.0000001401398
Correctly computed : 338
SNR :  999.9999999357101
Correctly computed : 339
SNR :  999.9999999515975
Correctly computed : 340
SNR :  1000.0000004529376
Correctly computed : 341
SNR :  1000.0000001392638
Correctly computed : 342
SNR :  1000.0000000270669
Correctly computed : 343
SNR :  1000.0000007396774
Correctly computed : 344
SNR :  1000.0000001100753
Correctly computed : 345
SNR :  1000.0000006185173
Correctly computed : 346
SNR :  999.9999998693208
Correctly computed : 347
SNR :  1000.0000006707664
Correctly computed : 348
SNR :  999.9999999726567
Correctly computed : 349
SNR :  1000.0000012406138
Correctly computed : 350
SNR :  1000.0000004403122
Correctly computed : 351
SNR :  999.999

Correctly computed : 499
SNR :  1000.0000005334499
Correctly computed : 500
SNR :  1000.000000383353
Correctly computed : 501
SNR :  1000.0000002150769
Correctly computed : 502
SNR :  1000.0000001622552
Correctly computed : 503
SNR :  1000.0000005903689
Correctly computed : 504
SNR :  1000.0000004975743
Correctly computed : 505
SNR :  999.999999985289
Correctly computed : 506
SNR :  999.9999998873349
Correctly computed : 507
SNR :  999.9999995507876
Correctly computed : 508
SNR :  1000.0000000571748
Correctly computed : 509
SNR :  999.9999997998959
Correctly computed : 510
SNR :  1000.0000002170284
Correctly computed : 511
SNR :  1000.0000001849297
Correctly computed : 512
SNR :  1000.0000003649545
Correctly computed : 513
SNR :  1000.0000000567463
Correctly computed : 514
SNR :  1000.0000003996182
Correctly computed : 515
SNR :  1000.0000006372434
Correctly computed : 516
SNR :  1000.0000006625005
Correctly computed : 517
SNR :  999.9999992878069
Correctly computed : 518
SNR :  1000.0

Correctly computed : 680
SNR :  1000.0000006101722
Correctly computed : 681
SNR :  999.9999994697052
Correctly computed : 682
SNR :  999.9999998339708
Correctly computed : 683
SNR :  1000.0000003712474
Correctly computed : 684
SNR :  1000.0000000771555
Correctly computed : 685
SNR :  999.9999996970508
Correctly computed : 686
SNR :  1000.0000002685165
Correctly computed : 687
SNR :  999.9999995497143
Correctly computed : 688
SNR :  1000.000000096591
Correctly computed : 689
SNR :  1000.0000000728634
Correctly computed : 690
SNR :  1000.0000002749657
Correctly computed : 691
SNR :  1000.0000007225619
Correctly computed : 692
SNR :  1000.000000068246
Correctly computed : 693
SNR :  999.9999998576666
Correctly computed : 694
SNR :  999.99999970706
Correctly computed : 695
SNR :  1000.000000099547
Correctly computed : 696
SNR :  1000.0000003639126
Correctly computed : 697
SNR :  1000.0000002809039
Correctly computed : 698
SNR :  999.9999998929868
Correctly computed : 699
SNR :  999.9999997

...

SNR :  1000.0000004193457
Correctly computed : 4114
SNR :  1000.0000003322486
Correctly computed : 4115
SNR :  999.9999996757231
Correctly computed : 4116
SNR :  999.9999999392496
Correctly computed : 4117
SNR :  1000.0000003642439
Correctly computed : 4118
SNR :  1000.0000000122403
Correctly computed : 4119
SNR :  1000.0000002297677
Correctly computed : 4120
SNR :  1000.0000000951952
Correctly computed : 4121
SNR :  999.9999997445873
Correctly computed : 4122
SNR :  999.9999999262869
Correctly computed : 4123
SNR :  1000.0000003310263
Correctly computed : 4124
SNR :  1000.0000005231697
Correctly computed : 4125
SNR :  1000.0000000093271
Correctly computed : 4126
SNR :  1000.0000001099047
Correctly computed : 4127
SNR :  1000.000000386838
Correctly computed : 4128
SNR :  1000.0000002792057
Correctly computed : 4129
SNR :  1000.0000002466744
Correctly computed : 4130
SNR :  1000.0000005405755
Correctly computed : 4131
SNR :  999.9999993703227
Correctly computed : 4132
SNR :  1000.000000

SNR :  999.9999998888932
Correctly computed : 4288
SNR :  999.999999970772
Correctly computed : 4289
SNR :  1000.0000009067656
Correctly computed : 4290
SNR :  1000.000000219939
Correctly computed : 4291
SNR :  1000.0000004576458
Correctly computed : 4292
SNR :  1000.0000005013402
Correctly computed : 4293
SNR :  1000.0000008717873
Correctly computed : 4294
SNR :  999.9999996972188
Correctly computed : 4295
SNR :  1000.000000218932
Correctly computed : 4296
SNR :  1000.0000000354742
Correctly computed : 4297
SNR :  1000.0000001244033
Correctly computed : 4298
SNR :  999.9999998372225
Correctly computed : 4299
SNR :  1000.0000008888733
Correctly computed : 4300
SNR :  1000.000000078312
Correctly computed : 4301
SNR :  999.9999998277013
Correctly computed : 4302
SNR :  999.9999998288057
Correctly computed : 4303
SNR :  1000.0000002371811
Correctly computed : 4304
SNR :  1000.0000000882508
Correctly computed : 4305
SNR :  1000.0000002254397
Correctly computed : 4306
SNR :  1000.0000004329

Correctly computed : 4462
SNR :  1000.0000000144767
Correctly computed : 4463
SNR :  1000.0000004462061
Correctly computed : 4464
SNR :  1000.0000009928634
Correctly computed : 4465
SNR :  1000.0000003074884
Correctly computed : 4466
SNR :  1000.0000000703212
Correctly computed : 4467
SNR :  1000.0000002626145
Correctly computed : 4468
SNR :  1000.0000002509788
Correctly computed : 4469
SNR :  999.9999996560177
Correctly computed : 4470
SNR :  999.9999998426795
Correctly computed : 4471
SNR :  1000.0000000177979
Correctly computed : 4472
SNR :  999.9999993423904
Correctly computed : 4473
SNR :  1000.0000001014424
Correctly computed : 4474
SNR :  999.9999998872995
Correctly computed : 4475
SNR :  1000.0000000236611
Correctly computed : 4476
SNR :  1000.0000003448215
Correctly computed : 4477
SNR :  999.9999996944861
Correctly computed : 4478
SNR :  999.999999596322
Correctly computed : 4479
SNR :  999.9999999233451
Correctly computed : 4480
SNR :  1000.000000083884
Correctly computed : 

Correctly computed : 4637
SNR :  1000.0000000846612
Correctly computed : 4638
SNR :  999.9999999037669
Correctly computed : 4639
SNR :  1000.0000002964878
Correctly computed : 4640
SNR :  1000.0000002264029
Correctly computed : 4641
SNR :  1000.0000001716924
Correctly computed : 4642
SNR :  999.9999999893146
Correctly computed : 4643
SNR :  1000.000000537133
Correctly computed : 4644
SNR :  1000.0000003425749
Correctly computed : 4645
SNR :  1000.0000007179563
Correctly computed : 4646
SNR :  1000.0000001475876
Correctly computed : 4647
SNR :  999.9999999654153
Correctly computed : 4648
SNR :  1000.0000003199475
Correctly computed : 4649
SNR :  1000.0000009411558
Correctly computed : 4650
SNR :  1000.0000004759781
Correctly computed : 4651
SNR :  999.9999992502673
Correctly computed : 4652
SNR :  999.9999998289409
Correctly computed : 4653
SNR :  1000.0000005934156
Correctly computed : 4654
SNR :  999.999999385238
Correctly computed : 4655
SNR :  999.9999997875082
Correctly computed : 

SNR :  999.9999994401107
Correctly computed : 4809
SNR :  1000.000000032633
Correctly computed : 4810
SNR :  999.9999998426795
Correctly computed : 4811
SNR :  1000.0000001138038
Correctly computed : 4812
SNR :  1000.0000000104525
Correctly computed : 4813
SNR :  999.9999999256819
Correctly computed : 4814
SNR :  1000.0000001760712
Correctly computed : 4815
SNR :  1000.0000001242886
Correctly computed : 4816
SNR :  1000.0000002840754
Correctly computed : 4817
Removed percentage: 0.0


In [36]:
# Create train and validation set with random SNR

x_train, y_train, prob = extract_features(train_files, y_train, num_ceps)
print('Removed percentage:', prob / len(y_train))
x_val, y_val, prob = extract_features(val_files, y_val, num_ceps)
print('Removed percentage:', prob / len(y_val))


Correctly computed : 0
Correctly computed : 1
Correctly computed : 2
Correctly computed : 3
Correctly computed : 4
Correctly computed : 5
Correctly computed : 6
Correctly computed : 7
Correctly computed : 8
Correctly computed : 9
Correctly computed : 10
Correctly computed : 11
Correctly computed : 12
Correctly computed : 13
Correctly computed : 14
Correctly computed : 15
Correctly computed : 16
Correctly computed : 17
Correctly computed : 18
Correctly computed : 19
Correctly computed : 20
Correctly computed : 21
Correctly computed : 22
Correctly computed : 23
Correctly computed : 24
Correctly computed : 25
Correctly computed : 26
Correctly computed : 27
Correctly computed : 28
Correctly computed : 29
Correctly computed : 30
Correctly computed : 31
Correctly computed : 32
Correctly computed : 33
Correctly computed : 34
Correctly computed : 35
Correctly computed : 36
Correctly computed : 37
Correctly computed : 38
Correctly computed : 39
Correctly computed : 40
Correctly computed : 41
Co

Correctly computed : 336
Correctly computed : 337
Correctly computed : 338
Correctly computed : 339
Correctly computed : 340
Correctly computed : 341
Correctly computed : 342
Correctly computed : 343
Correctly computed : 344
Correctly computed : 345
Correctly computed : 346
Correctly computed : 347
Correctly computed : 348
Correctly computed : 349
Correctly computed : 350
Correctly computed : 351
Correctly computed : 352
Correctly computed : 353
Correctly computed : 354
Correctly computed : 355
Correctly computed : 356
Correctly computed : 357
Correctly computed : 358
Correctly computed : 359
Correctly computed : 360
Correctly computed : 361
Correctly computed : 362
Correctly computed : 363
Correctly computed : 364
Correctly computed : 365
Correctly computed : 366
Correctly computed : 367
Correctly computed : 368
Correctly computed : 369
Correctly computed : 370
Correctly computed : 371
Correctly computed : 372
Correctly computed : 373
Correctly computed : 374
Correctly computed : 375


Correctly computed : 676
Correctly computed : 677
Correctly computed : 678
Correctly computed : 679
Correctly computed : 680
Correctly computed : 681
Correctly computed : 682
Correctly computed : 683
Correctly computed : 684
Correctly computed : 685
Correctly computed : 686
Correctly computed : 687
Correctly computed : 688
Correctly computed : 689
Correctly computed : 690
Correctly computed : 691
Correctly computed : 692
Correctly computed : 693
Correctly computed : 694
Correctly computed : 695
Correctly computed : 696
Correctly computed : 697
Correctly computed : 698
Correctly computed : 699
Correctly computed : 700
Correctly computed : 701
Correctly computed : 702
Correctly computed : 703
Correctly computed : 704
Correctly computed : 705
Correctly computed : 706
Correctly computed : 707
Correctly computed : 708
Correctly computed : 709
Correctly computed : 710
Correctly computed : 711
Correctly computed : 712
Correctly computed : 713
Correctly computed : 714
Correctly computed : 715


Correctly computed : 1010
Correctly computed : 1011
Correctly computed : 1012
Correctly computed : 1013
Correctly computed : 1014
Correctly computed : 1015
Correctly computed : 1016
Correctly computed : 1017
Correctly computed : 1018
Correctly computed : 1019
Correctly computed : 1020
Correctly computed : 1021
Correctly computed : 1022
Correctly computed : 1023
Correctly computed : 1024
Correctly computed : 1025
Correctly computed : 1026
Correctly computed : 1027
Correctly computed : 1028
Correctly computed : 1029
Correctly computed : 1030
Correctly computed : 1031
Correctly computed : 1032
Correctly computed : 1033
Correctly computed : 1034
Correctly computed : 1035
Correctly computed : 1036
Correctly computed : 1037
Correctly computed : 1038
Correctly computed : 1039
Correctly computed : 1040
Correctly computed : 1041
Correctly computed : 1042
Correctly computed : 1043
Correctly computed : 1044
Correctly computed : 1045
Correctly computed : 1046
Correctly computed : 1047
Correctly co

Correctly computed : 1357
Correctly computed : 1358
Correctly computed : 1359
Correctly computed : 1360
Correctly computed : 1361
Correctly computed : 1362
Correctly computed : 1363
Correctly computed : 1364
Correctly computed : 1365
Correctly computed : 1366
Correctly computed : 1367
Correctly computed : 1368
Correctly computed : 1369
Correctly computed : 1370
Correctly computed : 1371
Correctly computed : 1372
Correctly computed : 1373
Correctly computed : 1374
Correctly computed : 1375
Correctly computed : 1376
Correctly computed : 1377
Correctly computed : 1378
Correctly computed : 1379
Correctly computed : 1380
Correctly computed : 1381
Correctly computed : 1382
Correctly computed : 1383
Correctly computed : 1384
Correctly computed : 1385
Correctly computed : 1386
Correctly computed : 1387
Correctly computed : 1388
Correctly computed : 1389
Correctly computed : 1390
Correctly computed : 1391
Correctly computed : 1392
Correctly computed : 1393
Correctly computed : 1394
Correctly co

...

Correctly computed : 3175
Correctly computed : 3176
Correctly computed : 3177
Correctly computed : 3178
Correctly computed : 3179
Correctly computed : 3180
Correctly computed : 3181
Correctly computed : 3182
Correctly computed : 3183
Correctly computed : 3184
Correctly computed : 3185
Correctly computed : 3186
Correctly computed : 3187
Correctly computed : 3188
Correctly computed : 3189
Correctly computed : 3190
Correctly computed : 3191
Correctly computed : 3192
Correctly computed : 3193
Correctly computed : 3194
Correctly computed : 3195
Correctly computed : 3196
Correctly computed : 3197
Correctly computed : 3198
Correctly computed : 3199
Correctly computed : 3200
Correctly computed : 3201
Correctly computed : 3202
Correctly computed : 3203
Correctly computed : 3204
Correctly computed : 3205
Correctly computed : 3206
Correctly computed : 3207
Correctly computed : 3208
Correctly computed : 3209
Correctly computed : 3210
Correctly computed : 3211
Correctly computed : 3212
Correctly co

Correctly computed : 3521
Correctly computed : 3522
Correctly computed : 3523
Correctly computed : 3524
Correctly computed : 3525
Correctly computed : 3526
Correctly computed : 3527
Correctly computed : 3528
Correctly computed : 3529
Correctly computed : 3530
Correctly computed : 3531
Correctly computed : 3532
Correctly computed : 3533
Correctly computed : 3534
Correctly computed : 3535
Correctly computed : 3536
Correctly computed : 3537
Correctly computed : 3538
Correctly computed : 3539
Correctly computed : 3540
Correctly computed : 3541
Correctly computed : 3542
Correctly computed : 3543
Correctly computed : 3544
Correctly computed : 3545
Correctly computed : 3546
Correctly computed : 3547
Correctly computed : 3548
Correctly computed : 3549
Correctly computed : 3550
Correctly computed : 3551
Correctly computed : 3552
Correctly computed : 3553
Correctly computed : 3554
Correctly computed : 3555
Correctly computed : 3556
Correctly computed : 3557
Correctly computed : 3558
Correctly co

Correctly computed : 3873
Correctly computed : 3874
Correctly computed : 3875
Correctly computed : 3876
Correctly computed : 3877
Correctly computed : 3878
Correctly computed : 3879
Correctly computed : 3880
Correctly computed : 3881
Correctly computed : 3882
Correctly computed : 3883
Correctly computed : 3884
Correctly computed : 3885
Correctly computed : 3886
Correctly computed : 3887
Correctly computed : 3888
Correctly computed : 3889
Correctly computed : 3890
Correctly computed : 3891
Correctly computed : 3892
Correctly computed : 3893
Correctly computed : 3894
Correctly computed : 3895
Correctly computed : 3896
Correctly computed : 3897
Correctly computed : 3898
Correctly computed : 3899
Correctly computed : 3900
Correctly computed : 3901
Correctly computed : 3902
Correctly computed : 3903
Correctly computed : 3904
Correctly computed : 3905
Correctly computed : 3906
Correctly computed : 3907
Correctly computed : 3908
Correctly computed : 3909
Correctly computed : 3910
Correctly co

Correctly computed : 4222
Correctly computed : 4223
Correctly computed : 4224
Correctly computed : 4225
Correctly computed : 4226
Correctly computed : 4227
Correctly computed : 4228
Correctly computed : 4229
Correctly computed : 4230
Correctly computed : 4231
Correctly computed : 4232
Correctly computed : 4233
Correctly computed : 4234
Correctly computed : 4235
Correctly computed : 4236
Correctly computed : 4237
Correctly computed : 4238
Correctly computed : 4239
Correctly computed : 4240
Correctly computed : 4241
Correctly computed : 4242
Correctly computed : 4243
Correctly computed : 4244
Correctly computed : 4245
Correctly computed : 4246
Correctly computed : 4247
Correctly computed : 4248
Correctly computed : 4249
Correctly computed : 4250
Correctly computed : 4251
Correctly computed : 4252
Correctly computed : 4253
Correctly computed : 4254
Correctly computed : 4255
Correctly computed : 4256
Correctly computed : 4257
Correctly computed : 4258
Correctly computed : 4259
Correctly co

Correctly computed : 4572
Correctly computed : 4573
Correctly computed : 4574
Correctly computed : 4575
Correctly computed : 4576
Correctly computed : 4577
Correctly computed : 4578
Correctly computed : 4579
Correctly computed : 4580
Correctly computed : 4581
Correctly computed : 4582
Correctly computed : 4583
Correctly computed : 4584
Correctly computed : 4585
Correctly computed : 4586
Correctly computed : 4587
Correctly computed : 4588
Correctly computed : 4589
Correctly computed : 4590
Correctly computed : 4591
Correctly computed : 4592
Correctly computed : 4593
Correctly computed : 4594
Correctly computed : 4595
Correctly computed : 4596
Correctly computed : 4597
Correctly computed : 4598
Correctly computed : 4599
Correctly computed : 4600
Correctly computed : 4601
Correctly computed : 4602
Correctly computed : 4603
Correctly computed : 4604
Correctly computed : 4605
Correctly computed : 4606
Correctly computed : 4607
Correctly computed : 4608
Correctly computed : 4609
Correctly co

In [37]:
# Save MFCCs and their target

feature_sets_file = 'all_stored_files_targets_SNR.npz'
np.savez(feature_sets_file, 
         x_train=x_train, 
         y_train=y_train, 
         x_val=x_val, 
         y_val= y_val)



In [38]:
# Check if everything worked correctly 

feature_sets = np.load(feature_sets_file)
feature_sets.files

['x_train', 'y_train', 'x_val', 'y_val']